In [1]:
import numpy
import numpy as np
import time
import datetime
import netCDF4
import matplotlib.pylab as plt
import os,sys,glob
sys.path.append(os.getcwd()+'/../adjust/rasotools/')
import rasotools
from utils import *
from multiprocessing import Pool
#import odb
from eccodes import *
from functools import partial
from collections import OrderedDict
import subprocess
import json
import gzip
# from retrieve_fb_jra55 import add_feedback
import copy
import pickle
import xarray as xr

import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
matplotlib.rcParams.update({'font.size': 20})

plt.rcParams['lines.linewidth'] = 3

import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua

import zarr
import dask
from timeit import default_timer as timer


In [2]:
files = glob.glob('/users/staff/leo/fastscratch/rise/1.0/exp03/*/feedbackmerged*')

In [3]:
xr.open_dataset(files[0]).to_dataframe()

press      datum  temperatures  era5_fgdep  \
hour pressure time  station                                                
0    0        0     0          10.0 1948-01-01           NaN         NaN   
              1     0          10.0 1948-01-02           NaN         NaN   
              2     0          10.0 1948-01-03           NaN         NaN   
              3     0          10.0 1948-01-04           NaN         NaN   
              4     0          10.0 1948-01-05           NaN         NaN   
...                             ...        ...           ...         ...   
12   15       27019 0        1000.0 2022-01-02    284.950012         NaN   
              27020 0        1000.0 2022-01-03    283.350006         NaN   
              27021 0        1000.0 2022-01-04    279.149994         NaN   
              27022 0        1000.0 2022-01-07    280.950012         NaN   
              27023 0        1000.0 2022-01-08           NaN         NaN   

                             bias_estimate    lat    lon  hours  
hour pressure time  station                                      
0    0        0     0                  0.0  51.93 -10.25      3  
              1     0                  0.0  51.93 -10.25      3  
              2     0                  0.0  51.93 -10.25      3  
              3     0                  0.0  51.93 -10.25      3  
              4     0                  0.0  51.93 -10.25      3  
...                                    ...    ...    ...    ...  
12   15       27019 0                  0.0  51.93 -10.25     12  
              27020 0                  0.0  51.93 -10.25     12  
              27021 0                  0.0  51.93 -10.25     12  
              27022 0                  0.0  51.93 -10.25     12  
              27023 0                  0.0  51.93 -10.25     12  

[864768 rows x 8 columns]

In [5]:
rasotools.utils.

AttributeError: module 'rasotools' has no attribute 'utils'